In [ ]:
import kitchen.ingredients as k
import dropkick as dk
from MILWRM.ST import *
from MILWRM.MxIF import *
import sys; sys.path.append("../../../utility_scripts/")
from skimage import io
from pylab import *

# some stuff to make this notebook work better with Scanpy
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# make output directories
import os
if not os.path.exists("ST_out"):
    os.mkdir("ST_out")

if not os.path.exists("ST_out/plots_patient/"):
    os.mkdir("ST_out/plots_patient/")
    
if not os.path.exists("ST_out/plots_patient/wholeslide/"):
    os.mkdir("ST_out/plots_patient/wholeslide/")

In [ ]:
sc.set_figure_params(transparent=True, dpi_save=180)
sc.settings.figdir = "ST_out/plots_patient/wholeslide/"
import seaborn as sns; sns.set_style("white")

In [ ]:
sample_key = pd.read_csv("../resources/ST/visium_sample_key.csv", index_col=0)
sample_key.ws_adata = sample_key.ws_adata.fillna("")

---
# Whole-slide Patients

In [ ]:
refresh_obs = False

for ws in sample_key.loc[sample_key.ws_adata!="", "ws_adata"].unique():
    print("\nStarting {}".format(ws), end=" - ")
    sample_key_tmp = sample_key.loc[sample_key.ws_adata==ws, :].copy()
    print("detected {} constituent samples".format(len(sample_key_tmp)))
    mxif = sample_key_tmp.ws_npz[0]
    h_e = sample_key_tmp.ws_he[0]
    ws_path = sample_key_tmp.ws_adata[0]
    pat = sample_key_tmp.patient_name[0]
    block = sample_key_tmp.block_name[0]
    flip = sample_key_tmp.ws_flip[0]
    img0 = img.from_npz(file=mxif)
    img1 = io.imread(h_e)
    if img1.shape[:2] != img0.img.shape[:2]:
        img1 = img1[:img0.img.shape[0]:,:img0.img.shape[1]:,:]
    HE = img(img1, mask=img0.mask)
    HE.scale()
    if flip=="T":
        HE.img = HE.img[::-1,::-1,:]
        HE.mask = HE.mask[::-1,::-1]
    bg_img = HE.img
    for i in [0, 1, 2]:  # for 3-channel image
        bg_img[:, :, i][HE.mask == 0] = 1.0  # area outside mask 1.0
    
    # read in combined adata
    a_ws = sc.read(ws_path)
    
    # refresh .obs
    if refresh_obs:
        del a_ws.obs
        adatas = []
        if len(sample_key_tmp) > 1:
            for i in sample_key_tmp.index:
                tmp = sc.read("../data/ST/{}_master.h5ad".format(i))
                adatas.append(tmp)
            a = adatas[0].concatenate(adatas[1:], fill_value=0)
        else:
            a = sc.read("../data/ST/{}_master.h5ad".format(sample_key_tmp.index[0]))
        a_ws.obs = a_ws.obs.merge(a.obs, left_index=True, right_index=True)
        # Rename and re-color some things for the embedding plot
        cmap = cm.get_cmap("plasma", 8)
        mw_colordict = dict(zip(["D0","D1","D2","D3","D4","D5","D6","D7"], [matplotlib.colors.rgb2hex(cmap(i)) for i in range(cmap.N)]))
        a_ws.obs["MILWRM Domain"] = a_ws.obs["MILWRM Domain"].astype("category")
        a_ws.uns["MILWRM Domain_colors"] = [mw_colordict[x] for x in a_ws.obs["MILWRM Domain"].cat.categories]
        
        a_ws.obs["pathology_annotation"].replace({"carcinoma_edge":"carcinoma_border"}, inplace=True)
        path_colordict = dict(zip(["adenoma","adenoma_border","carcinoma","carcinoma_border","lymphoid_follicle","normal_mucosa","smooth_muscle"], sns.color_palette("tab10",7).as_hex()))
        a_ws.obs["pathology_annotation"] = a_ws.obs["pathology_annotation"].astype("category")
        a_ws.uns["pathology_annotation_colors"] = [path_colordict[x] for x in a_ws.obs["pathology_annotation"].cat.categories]

        # loading uns dict back in from .npy file
        cnv_leiden_colordict = np.load("infercnv/cnv_leiden_colors_{}.npy".format(pat), allow_pickle='TRUE').item()
        a_ws.obs["cnv_leiden"] = a_ws.obs["cnv_leiden"].astype("category")
        a_ws.uns["cnv_leiden_colors"] = [cnv_leiden_colordict[x] for x in a_ws.obs.cnv_leiden.cat.categories]

        # loading uns dict back in from .npy file
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_{}.npy".format(pat), allow_pickle='TRUE').item()
        a_ws.obs["CNV clone"] = a_ws.obs["CNV clone"].astype("category")
        a_ws.uns["CNV clone_colors"] = [cnv_clone_colordict[x] for x in a_ws.obs["CNV clone"].cat.categories]
        
        a_ws.write(ws_path, compression="gzip")
    
    # make histology obs empty column
    a_ws.obs["Histology"] = np.nan
    
    refNMF_cols = a_ws.obs.columns[a_ws.obs.columns.str.endswith("_VUMCrefNMF30")]
    new_cols = [x.split("_")[0] for x in refNMF_cols]
    a_ws.obs.rename(columns=dict(zip(refNMF_cols, new_cols)), inplace=True)
    
    # plot some shit
    sc.pl.spatial(
        a_ws,
        basis="spatial_ws",
        color=["Histology","CNV score","CNV clone"],
        ncols=3,
        img=bg_img,
        spot_size=50,
        scale_factor=1,
        crop_coord=[0,bg_img.shape[1],0,bg_img.shape[0]],
        frameon=False,
        cmap="viridis",
        vmax=0.12,
        save="_{}_{}_cnv_progression.png".format(pat, block),
        na_in_legend=False,
    )
    #sc.pl.spatial(
    #    a_ws,
    #    basis="spatial_ws",
    #    color=["IES"],# "DDR1", "PAK4", "TGFBI", "DPEP1", "FIB3_VUMCrefNMF30", "MYE2_VUMCrefNMF30", "FIB2_VUMCrefNMF30"],
    #    ncols=4,
    #    img=bg_img,
    #    spot_size=50,
    #    scale_factor=1,
    #    crop_coord=[0,bg_img.shape[1],0,bg_img.shape[0]],
    #    frameon=False,
    #    cmap="viridis",
    #    save="_{}_{}_immexcl.png".format(pat, block),
    #)
    #sc.pl.spatial(
    #    a_ws,
    #    basis="spatial_ws",
    #    color=["T cell CD4", "T reg suppressive", "TL1_VUMCrefNMF30", "T cell CD8", "TL2_VUMCrefNMF30", "TL3_VUMCrefNMF30", "MYE4_VUMCrefNMF30", "MYE5_VUMCrefNMF30"],
    #    ncols=4,
    #    img=bg_img,
    #    spot_size=50,
    #    scale_factor=1,
    #    crop_coord=[0,bg_img.shape[1],0,bg_img.shape[0]],
    #    frameon=False,
    #    cmap="viridis",
    #    save="_{}_{}_imminf.png".format(pat, block),
    #)
    
    print("DONE.\n")

---
# Other Patients

In [ ]:
for i in sample_key.loc[sample_key.ws_adata=="", :].index:
    print("\nStarting {}".format(i))
    # read in combined adata
    a_ws = sc.read("../data/ST/{}_master.h5ad".format(i))

    # Rename and re-color some things for the embedding plot
    cmap = cm.get_cmap("plasma", 8)
    mw_colordict = dict(zip(["D0","D1","D2","D3","D4","D5","D6","D7"], [matplotlib.colors.rgb2hex(cmap(i)) for i in range(cmap.N)]))
    a_ws.obs["MILWRM Domain"] = a_ws.obs["MILWRM Domain"].astype("category")
    a_ws.uns["MILWRM Domain_colors"] = [mw_colordict[x] for x in a_ws.obs["MILWRM Domain"].cat.categories]
    a_ws.obs["pathology_annotation"].replace({"carcinoma_edge":"carcinoma_border"}, inplace=True)
    path_colordict = dict(zip(["adenoma","adenoma_border","carcinoma","carcinoma_border","lymphoid_follicle","normal_mucosa","smooth_muscle"], sns.color_palette("tab10",7).as_hex()))
    a_ws.obs["pathology_annotation"] = a_ws.obs["pathology_annotation"].astype("category")
    a_ws.uns["pathology_annotation_colors"] = [path_colordict[x] for x in a_ws.obs["pathology_annotation"].cat.categories]
    
    # loading uns dict back in from .npy file
    if a_ws.obs.Patient.unique()[0]=="HTA11_08622_A":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    elif a_ws.obs.Patient.unique()[0]=="HTA11_08622_B":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    else:
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_{}.npy".format(a_ws.obs.Patient.unique()[0]), allow_pickle='TRUE').item()
    a_ws.obs["CNV clone"] = a_ws.obs["CNV clone"].astype("category")
    a_ws.uns["CNV clone_colors"] = [cnv_clone_colordict[x] for x in a_ws.obs["CNV clone"].cat.categories]
    
    # make histology obs empty column
    a_ws.obs["Histology"] = np.nan
    
    # plot some shit
    sc.pl.spatial(
        a_ws,
        color=["Histology","CNV score","CNV clone"],
        ncols=3,
        size=1.8,
        frameon=False,
        cmap="viridis",
        vmax=0.12,
        save="_{}_cnv_progression.png".format(i),
        na_in_legend=False,
    )
    #sc.pl.spatial(
    #    a_ws,
    #    color=["IES"],# "DDR1", "PAK4", "TGFBI", "DPEP1", "FIB3_VUMCrefNMF30", "MYE2_VUMCrefNMF30", "FIB2_VUMCrefNMF30"],
    #    ncols=4,
    #    size=1.8,
    #    frameon=False,
    #    cmap="viridis",
    #    save="_{}_immexcl.png".format(i),
    #)
    #sc.pl.spatial(
    #    a_ws,
    #    color=["T cell CD4", "T reg suppressive", "TL1_VUMCrefNMF30", "T cell CD8", "TL2_VUMCrefNMF30", "TL3_VUMCrefNMF30", "MYE4_VUMCrefNMF30", "MYE5_VUMCrefNMF30"],
    #    ncols=4,
    #    size=1.8,
    #    frameon=False,
    #    cmap="viridis",
    #    save="_{}_imminf.png".format(i),
    #)
    
    print("DONE.\n")